In [2]:
import os

import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms

from shared_methods import all_labels, show_image_by_path
from simpsons_neural_network_2 import SimpsonsNet2

# Check if we can use Cuda

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

# device = "cpu" # uncomment if you want to use "cpu", currently cpu is faster than cuda (maybe because the NN is very little)
print(f"Using {device} device")

Using cuda device


# Load the model

In [7]:
run_id = "mz69lif4"

model = SimpsonsNet2()
model.to(device)
model.load_state_dict(torch.load(f"trained_models/simpsons_net_2_{run_id}_4.pth"))
model.eval()

SimpsonsNet2(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=25088, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=29, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

# Test the model

In [8]:
# Create a DataLoader for the test dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

## Single image

In [9]:
with torch.no_grad():  # Disable gradient calculation
    path = "data/test/461179.jpg"
    # input_tensor = image_to_tensor(path)
    input_tensor = transform(Image.open(path)).to(device)
    output = model(input_tensor)  # Run inference

    # output tensor has shape [1, 29]
    # one batch dimension for my single image and 29 class scores

    probabilities = F.softmax(output, dim=1)
    predicted_label_idx = torch.argmax(probabilities).item()

    print(f"I guess this is {all_labels[predicted_label_idx]}")
    show_image_by_path(path)

ValueError: expected 4D input (got 3D input)

## All images

### Inference method 1

In [10]:
root_dir = "data/test"
batch_size = 32

all_files = [f for f in os.listdir(root_dir) if f.endswith(".jpg")]
num_batches = len(all_files) // batch_size

with open(f"solutions/solution_no1_{run_id}.csv", "w") as output_file:
    output_file.write("Id,Category\n")

    # This loop iterates over the list of all test image files in batches. The step size is batch_size.
    for i in range(0, len(all_files), batch_size):
        print(f"Predicting batch {i // batch_size + 1} of {num_batches}")

        # taking files from i to (i + batch_size)
        batch_files = all_files[i:i + batch_size]
        batch_tensors = [transform(Image.open(os.path.join(root_dir, f))) for f in batch_files]
        batch_tensors = torch.stack(batch_tensors)

        outputs = model(batch_tensors.to(device))  # shape ([32, 29]) (batch_size and number of classes)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)  # shape ([32, 29)]
        predicted_indices = torch.argmax(probabilities, dim=1)  # shape ([32]) -> label for each element in the batch

        for j, predicted_index in enumerate(predicted_indices):
            output_file.write(f"{batch_files[j]},{all_labels[predicted_index.item()]}\n")


Predicting batch 1 of 29
Predicting batch 2 of 29
Predicting batch 3 of 29
Predicting batch 4 of 29
Predicting batch 5 of 29
Predicting batch 6 of 29
Predicting batch 7 of 29
Predicting batch 8 of 29
Predicting batch 9 of 29
Predicting batch 10 of 29
Predicting batch 11 of 29
Predicting batch 12 of 29
Predicting batch 13 of 29
Predicting batch 14 of 29
Predicting batch 15 of 29
Predicting batch 16 of 29
Predicting batch 17 of 29
Predicting batch 18 of 29
Predicting batch 19 of 29
Predicting batch 20 of 29
Predicting batch 21 of 29
Predicting batch 22 of 29
Predicting batch 23 of 29
Predicting batch 24 of 29
Predicting batch 25 of 29
Predicting batch 26 of 29
Predicting batch 27 of 29
Predicting batch 28 of 29
Predicting batch 29 of 29
Predicting batch 30 of 29


## Inference method 2

In [11]:
root_dir = "data/test"

output_file = open(f"solutions/solution_no2_{run_id}.csv", "w")
output_file.write("Id,Category\n")

idx = 0

for filename in os.listdir(root_dir):
    if filename.endswith(".jpg"):
        if idx % 10 == 0:
            print(f"Predicting file with index {idx}")
        idx += 1

        img_path = os.path.join(root_dir, filename)
        input_tensor = transform(Image.open(img_path)).to(device)
        output = model(input_tensor)

        probabilities = F.softmax(output, dim=1)
        predicted_label_idx = torch.argmax(probabilities).item()

        output_file.write(f"{filename},{all_labels[predicted_label_idx]}\n")

output_file.close()


Predicting file with index 0


ValueError: expected 4D input (got 3D input)